In [1]:
import cv2
import cv2.aruco as aruco
import numpy as np
import math

In [25]:
aruco_dict = {}

In [75]:
def findAruco(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    key = getattr(aruco,f'DICT_5X5_250')
    arucoDict = aruco.Dictionary_get(key)
    arucoPara = aruco.DetectorParameters_create()

    (corners,ids,rejected) = cv2.aruco.detectMarkers(img, arucoDict, parameters = arucoPara)

    print(ids)
    if len(corners) > 0:
        ids = ids.flatten()

        for (markerCorner,markerid) in zip(corners,ids):
            corners = markerCorner.reshape((4,2))
            (topLeft,topRight,bottomLeft,bottomRight) = corners

            topRight = (int(topRight[0]),int(topRight[1]))
            topLeft = (int(topLeft[0]),int(topLeft[1]))
            bottomLeft = (int(bottomLeft[0]),int(bottomLeft[1]))
            bottomRight = (int(bottomRight[0]),int(bottomRight[1]))

            cx = int((topLeft[0]+bottomRight[0])/2.0)
            cy = int((topLeft[1]+bottomRight[1])/2.0)

            cx1 = int((topLeft[0]+bottomLeft[0])/2.0)
            cy1 = int((topLeft[1]+bottomLeft[1])/2.0)

            #sliced_img = img[topLeft[1]:bottomLeft[1],bottomLeft[0]:bottomRight[0]]

            cv2.circle(img, (cx,cy),5,(255,0,0),-1)
            cv2.circle(img, (cx1,cy1),5,(255,0,0),-1)
            cv2.line(img,(cx,cy),(cx1,cy1),(255,0,0),2)

            slope = (cy1-cy)/(cx1-cx)
            angle = (math.atan(slope)) * 180 / math.pi
            cv2.putText(img,str(angle),(cx,cy+10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

            # Generating a rotation matrix 

            matrix = cv2.getRotationMatrix2D((cx,cy), angle, 1.0) 
            h,w,c = img.shape

  
            # Performing the affine transformation 

            rotated = cv2.warpAffine(img, matrix, (w, h))

            #cropped_img = rotated[topLeft[1]:bottomRight[1],topLeft[0]:bottomRight[0]]
            #cropped_img = rotated[99:513,99:513]

            aruco_dict[markerid] = rotated
            #aruco_dict[markerid] = cropped_img

            cv2.imshow("image",img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
            #cv2.imshow("Rotated image",cropped_img)
            cv2.imshow("Rotated image",rotated)
            cv2.waitKey(0)
            cv2.destroyAllWindows()


In [76]:
img_a = cv2.imread(r"C:\Users\HP\OneDrive\Desktop\OpenCV_Project_Spriha\Ha.jpg")
img_b = cv2.imread(r"C:\Users\HP\OneDrive\Desktop\OpenCV_Project_Spriha\HaHa.jpg")
img_c = cv2.imread(r"C:\Users\HP\OneDrive\Desktop\OpenCV_Project_Spriha\LMAO.jpg")
img_d = cv2.imread(r"C:\Users\HP\OneDrive\Desktop\OpenCV_Project_Spriha\XD.jpg")

findAruco(img_a)
findAruco(img_b)
findAruco(img_c)
findAruco(img_d)

[[3]]
[[4]]
[[1]]
[[2]]


In [5]:
cropped_aruco_dict = {}

#SCREEN_DIMENSIONS = (591,591)

#def to_pixel_coords(relative_coords):
 #   return tuple(round(coord * dimension) for coord, dimension in zip(relative_coords, SCREEN_DIMENSIONS))

In [39]:
def crop_aruco(img2):
    gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    key = getattr(aruco,f'DICT_5X5_250')
    arucoDict = aruco.Dictionary_get(key)
    arucoPara = aruco.DetectorParameters_create()

    (corners,ids,rejected) = cv2.aruco.detectMarkers(img2, arucoDict, parameters = arucoPara)

    print(ids)
    if len(corners) > 0:
        ids = ids.flatten()

        for (markerCorner,markerid) in zip(corners,ids):
            corners = markerCorner.reshape((4,2))
            (topLeft,topRight,bottomLeft,bottomRight) = corners

            topRight = (int(topRight[0]),int(topRight[1]))
            topLeft = (int(topLeft[0]),int(topLeft[1]))
            bottomLeft = (int(bottomLeft[0]),int(bottomLeft[1]))
            bottomRight = (int(bottomRight[0]),int(bottomRight[1]))

            #print(bottomLeft[0],bottomRight[0], topLeft[1],bottomLeft[1])
            #row = to_pixel_coords(coords_1)
            #column = to_pixel_coords(coords_2)

            sliced_img = img2[topLeft[1]:bottomRight[1],topLeft[0]:bottomRight[0]]

            cropped_aruco_dict[markerid] = sliced_img

            cv2.imshow("sliced",sliced_img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

In [40]:
crop_aruco(aruco_dict[1])
crop_aruco(aruco_dict[2])
crop_aruco(aruco_dict[3])
crop_aruco(aruco_dict[4])

[[1]]
521 106 154 568
[[2]]
535 70 13 479


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:967: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [16]:
img1 = cv2.imread(r"C:\Users\HP\OneDrive\Desktop\OpenCV_Project_Spriha\CVtask.jpg")
h,w,c = img1.shape
print("Original Width and Height:", w,"x", h)

resized_img = cv2.resize(img1, (877,620))
cv2.imshow("Original",resized_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Original Width and Height: 1754 x 1240


In [54]:


def detect_Coloured_Squares(lower_bound,upper_bound):

    mask = cv2.inRange(resized_img, lower_bound, upper_bound)

    kernel = np.ones((7,7),np.uint8)
    # Remove unnecessary noise from mask
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    contours, hierarchy = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    segmented_img = cv2.bitwise_and(resized_img, resized_img, mask=mask)

    img2 = segmented_img.copy()

    l = 4
    for cnt in contours:
        if cv2.contourArea(cnt) > 1000:
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,peri*0.04,True)
            if len(approx) == l:
            
                x,y,w,h = cv2.boundingRect(cnt)
                ar = w / float(h)

                if ar >= 0.95 and ar <= 1.05:
                    rect = cv2.minAreaRect(cnt)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    cv2.drawContours(img2,[box],-1,(255,0,0),4)
                    cv2.putText(img2,f" {len(cnt)}",(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,-1,(0,0,0),2)
                    #cv2.rectangle(img2,(x,y),(x+w,y+h),(0,0,255),2)
                    print(x,y,w,h)
                    print(rect,box)

    cv2.imshow("image_contours",img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [55]:
# lower bound and upper bound for Green color
lower_bound_green = np.array([50, 20, 20])   
upper_bound_green = np.array([100, 255, 255])

detect_Coloured_Squares(lower_bound_green,upper_bound_green)


54 19 245 245
((176.18589782714844, 140.789306640625), (205.82113647460938, 205.8515167236328), 13.47315788269043) [[ 52 216]
 [100  16]
 [300  64]
 [252 264]]


In [57]:
# lower bound and upper bound for Orange color
lower_bound_orange = np.array([1,20,20])   
upper_bound_orange = np.array([18, 255, 255])

detect_Coloured_Squares(lower_bound_orange,upper_bound_orange)

586 39 175 175
((673.0, 126.0), (174.0, 174.0), 90.0) [[586  39]
 [760  39]
 [760 213]
 [586 213]]


In [58]:
# lower bound and upper bound for Black color
lower_bound_black = np.array([0, 0, 0])   
upper_bound_black = np.array([1, 1, 1])

detect_Coloured_Squares(lower_bound_black,upper_bound_black)

649 238 189 190
((742.3639526367188, 332.4992370605469), (148.48709106445312, 148.56173706054688), 22.496471405029297) [[645 372]
 [702 235]
 [839 292]
 [782 429]]


In [81]:
# lower bound and upper bound for Pink Peach color
lower_bound_pinkpeach = np.array([255, 219, 230])   
upper_bound_pinkpeach = np.array([255, 240, 240])

detect_Coloured_Squares(lower_bound_pinkpeach,upper_bound_pinkpeach)

In [53]:
resize1 = cv2.resize(aruco_dict[1],(200,200))
gray = cv2.cvtColor(resize1,cv2.COLOR_BGR2GRAY)
imgcanny = cv2.Canny(gray,30,150)
cv2.imshow("canny",imgcanny)
cv2.waitKey(0)
cv2.destroyAllWindows()

#large_img = segmented_img.copy()

large_img[216:64,52:300] = imgcanny
cv2.imshow("large",large_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

ValueError: could not broadcast input array from shape (200,200) into shape (0,248,3)

In [ ]:
bitand = cv2.bitwise_and(segmented_img_green,aruco_dict[1])
cv2.imshow("output",bitand)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(np.max(bitand[220:600,220:1000]))
print(np.min(bitand[220:600,220:1000]))

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:214: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'
